In [1]:
import pandas as pd
import numpy as np
import gzip
import json
from sklearn.naive_bayes import BernoulliNB

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments_5.json.gz')

In [3]:
df.overall.describe()

count    10261.000000
mean         4.488744
std          0.894642
min          1.000000
25%          4.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: overall, dtype: float64

In [4]:
df['text'] = df['reviewText'].astype(str) + df['summary']


In [5]:
positive = ['excellent', 'recommend', 'great','perfect', 'nice','works','good','fantastic', 'best']
negative = ['cannot', 'not', 'OK', 'disappointing', 'crap', 'beware', 'distorts']

for key in positive:
    df[str(key)] = df.text.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [6]:
df['score'] = np.where(df['text'].str.contains
                           ('(excellent|recommend|perfect|nice|works|good|fantastic|best)',
                                        regex=True,case=False), 1, 0)

c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
positive_score = df[positive]
df['score'] = (df['score'] == 1)
target = df['score']

In [8]:
bnb = BernoulliNB()

bnb.fit(positive_score, target)

ypredict = bnb.predict(positive_score)

print("Out of {} points: there are {} negative statements.".format(
     positive_score.shape[0],
     (target != ypredict).sum()
))

Out of 10261 points: there are 1801 negative statements.


In [9]:
from sklearn.model_selection import train_test_split
X = positive_score
y = target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)
print(Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)

(8208, 9) (8208,)
(2053, 9) (2053,)
